# **LSTM Autoencoder**

Simple Neural Network is feed-forward wherein info information ventures just in one direction.i.e. the information passes from input layers to hidden layers finally to the output layers. Recurrent Neural Network is the advanced type to the traditional Neural Network. It makes use of sequential information. Unlike conventional networks, the output and input layers are dependent on each other. RNNs are called recurrent because they play out a similar undertaking for each component of an arrangement, with the yield being relied upon the past calculations.LSTM or Long Short Term Memory are a type of RNNs that is useful in learning order dependence in sequence prediction problems.

In this notebook, we will cover a simple Long Short Term Memory autoencoder with the help of Keras and python

## **What is an LSTM autoencoder?**


LSTM autoencoder is an encoder that makes use of LSTM encoder-decoder architecture to compress data using an encoder and decode it to retain original structure using a decoder.

## **About the dataset**

The dataset can be downloaded from the following [link](https://www.kaggle.com/pdquant/sp500-daily-19862018). It gives the daily closing price of the S&P index

## **Code Implementation With Keras**

### **Import libraries required for this project**

In [ ]:
!python -m pip install pip --upgrade --user -q
!python -m pip install numpy pandas seaborn matplotlib scipy sklearn statsmodels keras tensorflow torch --user -q

In [ ]:
import IPython
IPython.Application.instance().kernel.do_shutdown(True)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

### **Read the data**

In [ ]:
df = pd.read_csv('spx.csv', parse_dates=['date'], index_col='date')

### **Split the data**

In [ ]:
train_size = int(len(df) * 0.9)
test_size = len(df) - train_size
train, test = df.iloc[0:train_size], df.iloc[train_size:len(df)]
train.shape

### **Pre-Processing of Data**

We need to pre-process the training and test data using the standardscaler library imported from sklearn.

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler = scaler.fit(train[['close']])
train['close'] = scaler.transform(train[['close']])
test['close'] = scaler.transform(test[['close']])

### **Create a sequence with historical data**

Now we will split the time series data into subsequences and create a sequence of 30 days of historical data.

In [ ]:
def create_dataset(X, y, time_steps=1):
    X1, y1 = [], []
    for i in range(len(X) - time_steps):
        t = X.iloc[i:(i + time_steps)].values
        X1.append(t)
        y1.append(y.iloc[i + time_steps])
    return np.array(X1), np.array(y1)

In [ ]:
TIME_STEPS = 30
X_train, y_train = create_dataset(
  train[['close']],
  train.close,
  TIME_STEPS
)
X_test, y_test = create_dataset(
  test[['close']],
  test.close,
  TIME_STEPS
)
print(X_train.shape)

### **Creating an LSTM Autoencoder Network**

The architecture will produce the same sequence as given as input. It will take the sequence data. The dropout removes inputs to a layer to reduce overfitting. Adding RepeatVector to the layer means it repeats the input n number of times. The TimeDistibuted layer takes the information from the previous layer and creates a vector with a length of the output layers.

In [ ]:
import keras
model = keras.Sequential()
model.add(keras.layers.LSTM(
    units=64,
    input_shape=(X_train.shape[1], X_train.shape[2])
))
model.add(keras.layers.Dropout(rate=0.2))
model.add(keras.layers.RepeatVector(n=X_train.shape[1]))
model.add(keras.layers.LSTM(units=64, return_sequences=True))
model.add(keras.layers.Dropout(rate=0.2))
model.add(
  keras.layers.TimeDistributed(
    keras.layers.Dense(units=X_train.shape[2])
  )
)
model.compile(loss='mae', optimizer='adam')
model.summary()

### **Fitting the Model**

Here, we train the model with epoch:20 and batch size 32.

In [ ]:
history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=32,
    validation_split=0.1,
    shuffle=False
)

### **Evaluation**

In [ ]:
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend();

From the above plot we can see the training and test error is decreasing. For better result, we can train the model with more epochs.

### **Actual Value of Test Data**

In [ ]:
y_test

### **Prediction on Test Data**

In [ ]:
pred = model.predict(X_test, verbose=0)

# **Related Articles --**

> * [Introduction to LSTM Autoencoder](https://analyticsindiamag.com/introduction-to-lstm-autoencoder-using-keras/)
> * [Beginners Guide to MLP](https://analyticsindiamag.com/a-beginners-guide-to-scikit-learns-mlpclassifier/)
> * [Beginners Guide to PyTorch](https://analyticsindiamag.com/a-beginners-guide-pytorch/)
> * [Loss functions in PyTorch](https://analyticsindiamag.com/all-pytorch-loss-function/)
> * [Loss functions in Tensorflow Keras](https://analyticsindiamag.com/ultimate-guide-to-loss-functions-in-tensorflow-keras-api-with-python-implementation/)
> * [Loss function with examples](https://analyticsindiamag.com/loss-functions-in-deep-learning-an-overview/)
> * [Optimizers in Tensorflow Keras](https://analyticsindiamag.com/guide-to-tensorflow-keras-optimizers/)
> * [Deep Learning Frameworks](https://analyticsindiamag.com/deep-learning-frameworks/)
> * [Types of Activation Functions](https://analyticsindiamag.com/most-common-activation-functions-in-neural-networks-and-rationale-behind-it/)
> * [Maths for Deep Learning](https://analyticsindiamag.com/beginners-guide-neural-network-math-python/)
> * [Deep Learning Using Tensorflow Keras](https://analyticsindiamag.com/deep-learning-using-tensorflow-keras/)